# Returns a Malaria prevalence raster using a linear function

In [2]:
import rioxarray
import numpy

In [12]:
# Covariates
lst = rioxarray.open_rasterio("Uganda Standardized Rasters/standard_uganda_LSTday_2km_2018.tif")
rainfall = rioxarray.open_rasterio("Uganda Standardized Rasters/standard_uganda_Rainfall_CHIRPS_2km_2018.tif")
elevation = rioxarray.open_rasterio("Uganda Standardized Rasters/standard_uganda_elevation_2km.tif")

In [9]:
lst = lst.sel(band=1)

In [10]:
lst

<xarray.DataArray (y: 343, x: 328)> Size: 450kB
array([[ 0.754582,  0.523065,  0.653057, ...,  2.017498,  2.004043,  1.99516 ],
       [ 0.842711,  0.712415,  0.715094, ...,  1.987648,  1.8884  ,  1.932979],
       [ 0.46912 ,  0.517618,  0.937721, ...,  1.900321,  1.944083,  1.990718],
       ...,
       [-1.011001, -1.033208, -1.12648 , ...,  0.921055,  0.987006,  1.203712],
       [-0.702316, -0.933805, -1.004339, ...,  0.862421,  1.026941,  1.101591],
       [-0.751172, -0.663823, -0.926612, ...,  0.997289,  1.098021,  0.990341]],
      dtype=float32)
Coordinates:
    band         int64 8B 1
  * x            (x) float64 3kB 29.58 29.6 29.62 29.63 ... 35.0 35.02 35.03
  * y            (y) float64 3kB 4.225 4.208 4.192 ... -1.442 -1.458 -1.475
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [4]:
# Reprojects covariates
elevation = elevation.rio.reproject_match(lst)

In [5]:
elevation = elevation.assign_coords({
    "x": lst.x,
    "y": lst.y
})

In [176]:
# Arbitrary function
malaria_logit_prevalence = (-1.5 + .5 * lst + .75 * rainfall - .25 * elevation)

In [177]:
malaria_prevalence_probability = 1 / (1 + numpy.exp(-malaria_logit_prevalence))

In [178]:
malaria_prevalence_probability.rio.to_raster("Uganda Malaria Data/uganda_mock_malaria_prevelance_2km_2018.tif")